In [8]:
%matplotlib notebook

# loading standard modules
import numpy as np
import matplotlib.pyplot as plt
import maxflow
from skimage import img_as_ubyte
from sklearn import mixture
from skimage.color import rgb2gray
# loading custom module (requires file asg1.py in the same directory as the notebook file)
from asg1_error_handling import Figure, GraphCutsPresenter

In [4]:

def wpq(lambda_, p, q, sigma):
  return lambda_ * np.exp(- ((p - q) ** 2) / sigma ** 2)

def wpq_vector(lambda_, p, q, sigma): 
    return lambda_ * np.exp(-(np.linalg.norm(p - q, axis = 2) ** 2) / sigma ** 2)

def wpq_vector2(lambda_, p, q, sigma): 
    d = p-q
    return lambda_ * np.exp(-((d[...,0]**2)+(d[...,1]**2)+(d[...,2]**2)) / sigma ** 2)


In [5]:
img1 = plt.imread('images/Meki.png')
wpq_vector(1, img1, np.roll(img1, -1, axis = 1), 5).shape

(512, 512)

In [6]:
class MyGraphCuts:
    bgr_value = 0
    obj_value = 1
    none_value = 2
    
    def __init__(self, img, sigma, lambda_):
        
        
        self.num_rows = img.shape[0]
        self.num_cols = img.shape[1]
        
        self.sigma = sigma
        self.img = img
        self.lambda_ = lambda_
        
        self.g = self.calculate_graph( img, lambda_, sigma)
        
        self.bgr_value = 0
        self.obj_value = 1
        
        
        
        # comment out the figrue class before debugging
        self.fig = Figure()
        self.pres = GraphCutsPresenter(img, self)
        self.pres.connect_figure(self.fig)

        '''
        self.g.add_grid_tedges(self.nodeids, 
                               wpq_vector2( self.lambda_, img, 57, self.sigma), 
                               wpq_vector2( self.lambda_, img, 300, self.sigma))
        '''
        
    def run(self):
        self.fig.show()
        

    

    
    
    
    
    

In [57]:
class MyGraphCutsCoulorBasedLikelihood:
    bgr_value = 0
    obj_value = 1
    none_value = 2
    
    def __init__(self, img, sigma, lambda_, bkg_intensity, obj_intensity):
        
        
        self.num_rows = img.shape[0]
        self.num_cols = img.shape[1]
        
        self.sigma = sigma
        self.img = img
        self.lambda_ = lambda_
        
        self.g = self.calculate_graph( img, lambda_, sigma)
        
        self.bgr_value = 0
        self.obj_value = 1
        
        
        
        # comment out the figrue class before debugging
        self.fig = Figure()
        self.pres = GraphCutsPresenter(img, self)
        self.pres.connect_figure(self.fig)

        self.red_model = mixture.GaussianMixture(n_components=6, covariance_type='full')
        
        self.blue_model = mixture.GaussianMixture(n_components=6, covariance_type='full')



        
    def run(self):
        self.fig.show()
        
    def calculate_graph(self, img, lambda_, sigma):
        #img = np.sum(img, axis = 2) / 3
        g = maxflow.Graph[float]()
        self.nodeids = g.add_grid_nodes((self.num_rows, self.num_cols))
        nodeids = self.nodeids
        def wpq(lambda_, p, q, sigma):
            return lambda_ * np.exp(- ((p - q) ** 2) / sigma ** 2)
        
        n_right =  wpq_vector(lambda_, img, np.roll(img, -1, axis = 1), sigma)  
        n_below =  wpq_vector(lambda_, img, np.roll(img, -1, axis = 0), sigma) 


        structure_x = np.array([[0, 0, 0],
                                [0, 0, 1],
                                [0, 0, 0]])
        structure_y = np.array([[0, 0, 0],
                                [0, 0, 0],
                                [0, 1, 0]])

        g.add_grid_edges(nodeids, weights = n_right, structure = structure_x, symmetric = True)
        g.add_grid_edges(nodeids, weights = n_below, structure = structure_y, symmetric = True)
        return g
    
    def compute_labels(self, seed_mask):
        num_rows = self.num_rows
        num_cols = self.num_cols
        img = self.img
        # +---------+---------+
        # |         |         |
        # |   bgr   |  none   |
        # |         |         |
        # +---------+---------+
        # |         |         |
        # |  none   |   obj   |
        # |         |         |
        # +---------+---------+
        
        blue_points = img[seed_mask==self.obj_value]
        red_points = img[seed_mask==self.bgr_value]
        
        print(blue_points.shape)
        
        # computing tlinks


        # set of colors for all image pixels
        R = img[:,:,0].flatten()
        G = img[:,:,1].flatten()
        B = img[:,:,2].flatten()

        like_red_im = np.zeros((num_rows, num_cols))
        like_blue_im = np.zeros((num_rows, num_cols))
        data  = np.vstack([R,G,B])
        if blue_points.size != 0:
            data_blue = np.column_stack((blue_points))
            self.blue_model.fit(data_red.T)
            like_blue = -self.blue_model.score_samples(data.T)
            like_blue_im = like_blue.reshape((num_rows, num_cols))
            
        if red_points.size != 0:
            data_red = np.column_stack((red_points))
            self.red_model.fit(data_red.T)
            like_red = -self.red_model.score_samples(data.T)
            like_red_im = like_red.reshape((num_rows, num_cols))
            
        
        
        self.g.add_grid_tedges(self.nodeids, like_red_im, like_blue_im, self.sigma)
        
        
        bgr_seed = np.where(seed_mask==self.bgr_value, self.lambda_ * 4, 0)
        obj_seed = np.where(seed_mask==self.obj_value, self.lambda_ * 4, 0)
        
        #bgr_seed = np.where(seed_mask==self.bgr_value, wpq_vector2( self.lambda_, img, 57, self.sigma), 0)
        #obj_seed = np.where(seed_mask==self.obj_value, wpq_vector2( self.lambda_, img, 100, self.sigma), 0)
        
        
        
        
        self.g.add_grid_tedges(self.nodeids, bgr_seed, obj_seed)
        
        self.g.maxflow()
        # Get the segments of the nodes in the grid.
        # sgm.shape == nodeids.shape
        sgm = self.g.get_grid_segments(self.nodeids)
        
        #segments = self.none_value * np.ones((num_rows ,num_cols ))
        #segments[sgm] = self.bgr_value
        #segments[sgm == False] = self.obj_value
        return sgm #segments
    
    
    
    
    

### Notes about the basic graph cut interface:
1. To provide the regional hard constraints (seeds) for object and background segments use left and right mouse clicks (mouse dragging works somewhat too). Use mouse wheel to change the brush size.
2. The seed mask is built by the "GraphCutsPresenter". Each mouse release activates "on_mouse_up" function of the presenter, which asks the linked MyGraphCuts object to "compute_labels" for all pixels
based on the provided seed mask.
3. You should use "PyMaxflow" library (already imported into this notebook if you ran the second cell) to build a weighted graph and to compute a minimum s/t cut defining all pixel labels from the seeds as explain in topic 5.

In [10]:
img1 = plt.imread('images/Meki.png')
app = MyGraphCuts(img1, 50, 10)
app.run()






<IPython.core.display.Javascript object>

In [11]:

img1 = plt.imread('images/bunny.bmp')
app = MyGraphCutsCoulorBasedLikelihood(img1, 50, 20, [1,2,3], [1,2,3])
app.run()






FileNotFoundError: [Errno 2] No such file or directory: 'images/bunny.bmp'

In [44]:
sigma = [0.01, 0.1, 1, 10, 100]
img1 = plt.imread('images/bunny.bmp')
print('Sigma used:', sigma[1])
app = MyGraphCuts(img1, sigma[0], 70, 40)
app.run()

Sigma used: 0.1


TypeError: __init__() takes 4 positional arguments but 5 were given

### Show how your interactive segmenter works on more challanging images where there is some overlap between the color-models in the object and background (as in the "lama" image). Compare the results for $\lambda=0$ (no regularization) and for some $\lambda>0$. For convenience, you might want to include $\lambda$ in the list of parameters for the function "MyGraphcuts".

The Case where $\lambda = 0$

In [ ]:
img2 = plt.imread('images/lama.jpg')
app = MyGraphCuts(img2, sigma = 1, lambda_ = 0)
app.run()

I really got no idea how graph cut works programatically I have looked at the slides and understands it conceptually. I cannot get the maxflow to work. Due to the nature of the assignment I am unable to use print statements as debug prompts. 

In [ ]:
img2 = plt.imread('images/lama.jpg')
app = MyGraphCuts(img2, sigma = 1, 3)
app.run()